In [1]:
import pyspark

sc = pyspark.sql.SparkSession.Builder().getOrCreate()

## Levanto los archivos

In [2]:
DIR = "datos/"
FILE_COORDENADAS = "DistancesCoordenadasUTM.csv"
FILE_CALIBRACION = "DatosCalibracion.csv"
FILE_RECEPTORES = ["DatosRC1.csv", "DatosRC2.csv", "DatosD1.csv", "DatosD2.csv"] 

def rdd_from_file(filename):
    return sc.read.csv(filename, header=True, inferSchema=True).rdd


cal_rdd = rdd_from_file(DIR+FILE_CALIBRACION)
recep_rdd = [rdd_from_file(DIR+name) for name in FILE_RECEPTORES]

## Utils

In [3]:
def apply_all(list_rdd, f):
    return list(map(f, list_rdd)) 

def remove_no_receptions(l):
    return filter(lambda e: e > 0,l)

## Filtro los datos de calibración vacios

In [13]:
cal_rdd = cal_rdd.filter(lambda x: x['Fecha'] != 'NA')

## Filtro datos de receptores invalidos (hay invalidos porque se repite el header)

In [4]:
recep_rdd = apply_all(recep_rdd, lambda recep: recep.filter(lambda row: row['Date'] is not None))


## Fix fechas ambiguas

In [5]:
import re
from datetime import datetime, timedelta

pattern = re.compile('[1-9][0-9]*/[1-9][0-9]*/.*')
global pattern

def fix_date_format(row, field):
    row_dict = row.asDict()
    date = row[field]
    format_from = '%m/%d/%Y'
    format_to = '%d/%m/%Y'
    if pattern.match(date):
        row_dict[field] = datetime.strptime(date, format_from).strftime(format_to)
    return row_dict

recep_rdd = apply_all(recep_rdd, lambda recep: recep.map(lambda x: fix_date_format(x, 'Date'))) 


## Agrego timestamp

In [6]:
def add_timestamp(row, date_field, time_field, suffix = ''):
    row_dict = row if isinstance(row, (dict)) else row.asDict()
    row_dict['timestamp'+suffix] = datetime.strptime(row[date_field]+' '+ row[time_field], '%d/%m/%Y %H:%M:%S')
    return row_dict
    
recep_rdd = apply_all(recep_rdd, lambda recep: recep.map(lambda x: add_timestamp(x, 'Date', 'Time'))) 



## Saco recepciones que no son de pajaros

In [14]:
cal_tags = cal_rdd.map(lambda x: int(x['Tag'])).distinct().collect()
birds_recep_rdd = apply_all(recep_rdd, lambda recep: recep.filter(lambda x: x['Tag ID'] not in cal_tags))

In [ ]:
'''
aux = datetime.strptime('28/01/2018 12:46:46', '%d/%m/%Y %H:%M:%S')
aux2 = aux.replace(second=0)
print(aux)
print(aux2)
'''
int(30.99)

## Agrupo emisiones cada 30 seg

In [21]:
bin_time = 30

def merge_recep(grouped_data):
    recepciones = grouped_data[1]
    return {
        'Tag': grouped_data[0][0],
        'timestamp': grouped_data[0][1],
        'recep': [recep['Power'] for recep in grouped_data[1]],
    }
    
#birds_recep_rdd[0].groupBy(lambda x: (x['Tag ID'], x['timestamp'].replace(second=int(x['timestamp'].second/bin_time)))).map(merge_recep).take(1)
birds_recep_rdd = apply_all(birds_recep_rdd, lambda recep: recep.groupBy(lambda x: (x['Tag ID'], x['timestamp'].replace(second=int(x['timestamp'].second/bin_time)))).map(merge_recep))


### Agrupo las 4 antenas

In [59]:
birds_recep_0_df = birds_recep_rdd[0].toDF().selectExpr('*', 'recep as recep_0').alias('recep')
birds_recep_1_df = birds_recep_rdd[1].toDF().alias('recep_1')
birds_recep_2_df = birds_recep_rdd[2].toDF().alias('recep_2')
birds_recep_3_df = birds_recep_rdd[3].toDF().alias('recep_3')



/Users/alew/py3/lib/python3.6/site-packages/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [61]:


merge1 = birds_recep_0_df.join(birds_recep_1_df, (birds_recep_0_df['Tag ID'] == birds_recep_1_df['Tag ID']) & (birds_recep_0_df.timestamp == birds_recep_1_df.timestamp), how='full')

    
    

In [64]:
cond_time = """case when recep.timestamp is not null then recep.timestamp
            else case when recep_1.timestamp is not null then recep_1.timestamp
                end
            end as timestamp"""
cond_tag = """case when recep.`Tag ID` is not null then recep.`Tag ID`
            else case when recep_1.`Tag ID` is not null then recep_1.`Tag ID`
                end
            end as `Tag ID`"""


merge1 = merge1.selectExpr(cond_tag, cond_time, 'recep.recep_0', 'recep_1.recep as recep_1').alias('recep')

In [65]:
merge2 = merge1.join(birds_recep_2_df, (merge1['Tag ID'] == birds_recep_2_df['Tag ID']) & (merge1.timestamp == birds_recep_2_df.timestamp), how='full')



In [77]:
merge2 = merge2.selectExpr(cond_tag.replace('recep_1', 'recep_2'), cond_time.replace('recep_1', 'recep_2'), 'recep.recep_0', 'recep.recep_1', 'recep_2.recep as recep_2').alias('recep')



In [78]:
merge2.take(1)

[Row(Tag ID=1, timestamp=datetime.datetime(2018, 2, 21, 7, 21), recep_0=[34], recep_1=None, recep_2=None)]

In [79]:
merge3 = merge2.join(birds_recep_3_df, (merge2['Tag ID'] == birds_recep_3_df['Tag ID']) & (merge2.timestamp == birds_recep_3_df.timestamp), how='full')




In [84]:
merge_all_df = merge3.selectExpr(cond_tag.replace('recep_1', 'recep_3'), cond_time.replace('recep_1', 'recep_3'), 'recep.recep_0', 'recep.recep_1', 'recep.recep_2', 'recep_3.recep as recep_3')



### Change null to empty list

In [85]:
def fix_null_recep(row):
    row_dict = row.asDict()
    for i in range(4):
        if row_dict['recep_{}'.format(i)] is None:
            row_dict['recep_{}'.format(i)] = []
    return row_dict
    

merge_all = merge_all_df.rdd.map(fix_null_recep)

In [87]:
import json

def my_converter(o):
    if isinstance(o, datetime):
        return o.__str__()
    

merge_all.map(lambda x: json.dumps(x, default=my_converter)).saveAsTextFile('datos/birds-recep.jsonlines')